In [ ]:
import os
from pydub import AudioSegment
import pympi
import pandas as pd
from sklearn.model_selection import train_test_split

eaf_files = [
    '/mnt/PROJET_CAENNAIS/corpus/GB/3_transcription/gold/GB_E1.eaf',
    "/mnt/PROJET_CAENNAIS/corpus/GB/3_transcription/gold/GB_E2.eaf",
    "/mnt/PROJET_CAENNAIS/corpus/GB/3_transcription/gold/GB_E3.eaf",
    "/mnt/PROJET_CAENNAIS/corpus/GC/3_transcription/gold/GC_E1.eaf"

]
wav_files = [
    '/home/ziane212/projects/MMS_ASR_finetuning/caennais_gold_12.11.24/E1_GB_WAV.wav',
    '/home/ziane212/projects/MMS_ASR_finetuning/caennais_gold_12.11.24/E2_GB_WAV_anon.wav'
]

output_dir = '/home/ziane212/projects/MMS_ASR_finetuning/caennais_gold_12.11.24/dataset_caennais_12.11.24/audio'

# Vérifiez que le nombre de fichiers EAF et WAV correspond
if len(eaf_files) != len(wav_files):
    raise ValueError("Le nombre de fichiers EAF et WAV doit être le même.")

# Préparer une liste pour stocker les données du tableau TSV
data = []

# Initialiser un compteur pour les annotations passées
annotations_passed = 0

# Parcourir chaque paire de fichiers EAF et WAV
for eaf_file, wav_file in zip(eaf_files, wav_files):
    # Extraire le nom du fichier sans extension pour l'utiliser dans les noms des extraits
    # file_base_name = os.path.splitext(os.path.basename(eaf_file))[0]
    file_base_name = os.path.basename(eaf_file).replace('.eaf', '')
    print(file_base_name)
    # Charger le fichier EAF
    eaf = pympi.Elan.Eaf(eaf_file)

    # Charger le fichier audio
    audio = AudioSegment.from_wav(wav_file)

    # Parcourir les tiers et les annotations
    for tier in eaf.tiers:
        for annotation in eaf.get_annotation_data_for_tier(tier):
            start_time, end_time, value = annotation
            # Vérifier la durée du segment
            duration = (end_time - start_time) / 1000  # Convertir en secondes
            value = value.rstrip(' ').lstrip(' ')
            if duration > 30:
                # Incrémenter le compteur si l'annotation est passée
                annotations_passed += 1
                continue
            elif value == "":
                continue
            elif value.startswith("[") and value.endswith("]"):
                continue
            elif len(value) <= 4:
                continue
            # Découper l'extrait audio selon les timecodes
            extract = audio[start_time:end_time]
            # Définir le chemin de sortie pour l'extrait, incluant le nom du fichier source
            output_file = os.path.join(output_dir, f"{file_base_name}_{tier}_{start_time}_{end_time}.wav")
            # Sauvegarder l'extrait
            extract.export(output_file, format="wav")
            # Ajouter les informations dans la liste des données
            data.append([output_file, value])

# Créer un DataFrame pandas pour le tableau TSV
df = pd.DataFrame(data, columns=["audio", "text"])

# Split data into train (80%) and test (20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Sauvegarder les tableaux en TSV
train_tsv_file = os.path.join(output_dir, "train_dataset_caennais_12.11.24.tsv")
test_tsv_file = os.path.join(output_dir, "test_dataset_caennais_12.11.24.tsv")
train_df.to_csv(train_tsv_file, sep='\t', index=False)
test_df.to_csv(test_tsv_file, sep='\t', index=False)

print(f"Process completed. Train and Test TSV files saved in {output_dir}.")
print(f"Number of annotations passed: {annotations_passed}")


E1_GB_WAV
E2_GB_WAV_26min
Process completed. Train and Test TSV files saved in /home/ziane212/projects/MMS_ASR_finetuning/caennais_gold_12.11.24/dataset_caennais_12.11.24/audio.
Number of annotations passed: 0
